### Cryptogue Week 3:Advanced Cryptographic Methods

#### Topics covered:
1. **Matrices and Lattice Reduction (LLL Algorithm)**
2. **Short Integer Solution (SIS)**
3. **Learning with Errors (LWE)**
4. **Knapsack Problem and Cryptanalysis**

#### Topics covered in this notebook will require extensive use of functions from the sagemath library. Using an online Sagemath interface or downloading the library directly is recommended. [Sage Online Server Cell](https://sagecell.sagemath.org/)

In [9]:
from sage.all import *
import numpy as np
import matplotlib.pyplot as plt

# 1. Matrices and Lattice Reduction (LLL Algorithm)
Lattices are discrete mathematical structures used in cryptography.

<p align="center">
  <img src="./images/lattice.png" alt="image not in the same folder" width = "500px">
</p>

LLL is a polynomial-time algorithm that finds a nearly orthogonal, short basis for a given lattice. Given a basis $ {b_1,b_2,...,b_n} $, the algorithm outputs the reduced basus $ {b_{1}^{'}, b_{2}^{'}, b_{n}^{'}} $ such that:
- The vectors are shorter than the original basis vectors.
- They are nearly orthogonal, meaning they are close to being perpendicular.
LLL is based on the **Gram-Schmidt**(remember MA110?) process but improves the basis iteratively using a **size reduction** and a **Lovász condition** to ensure quality.

<p align="center">
  <img src="./images/LLL.png" alt="image not in the same folder" width = "500px">
</p>

#### **USES:**
1. **Cryptanalysis:** Breaking RSA, Factoring lattices, attacking knapsack cryptosystems, breaking *elliptic curve cryptography*.
2. **Post-Quantum Cryptography**
   
#### **Brief Overview**
The algorithm maintains a Gram-Schmidt orthogonalization of the basis vectors while iteratively improving them by:
- Size reduction: Ensuring coefficients in Gram-Schmidt decomposition remain small.
- Lovász condition: Checking if the basis satisfies an optimality condition; if not, swap vectors and repeat.
  
It runs in polynomial time $O(n^5 log^3 B)$ where n is the lattice dimension and B is the bound on coefficients.

In [1]:
def plot_lattice(basis, title="Lattice Basis", color='b'):
    fig, ax = plt.subplots(figsize=(6, 6))

    grid_range = np.arange(-10, 10)
    for x in grid_range:
        for y in grid_range:
            point = x * basis[0] + y * basis[1]
            ax.scatter(*point, color='gray', s=10, alpha=0.5)

    origin = np.array([0, 0])
    ax.quiver(*origin, *basis[0], color=color, angles='xy', scale_units='xy', scale=1, width=0.006)
    ax.quiver(*origin, *basis[1], color=color, angles='xy', scale_units='xy', scale=1, width=0.006)

    ax.axhline(0, color='black', linewidth=1)
    ax.axvline(0, color='black', linewidth=1)
    ax.set_xlim(-20, 20)
    ax.set_ylim(-20, 20)
    ax.set_xticks(range(-20, 21, 5))
    ax.set_yticks(range(-20, 21, 5))
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.set_title(title)
    plt.show()

In [ ]:
B = Matrix(ZZ, [[4, 1], [7, 4]])
print("Original Basis:")
print(B)
plot_lattice(B,"Lattice Basis before LLL")

In [ ]:
B_lll = B.LLL()
print("LLL Reduced Basis:")
print(B_lll)
plot_lattice(B_lll, "Lattice Basis after LLL")

# 2. Short Integer Solution (SIS)

The **Short Integer Solution (SIS)** problem is a **fundamental lattice-based hardness assumption** used in post-quantum cryptography. It is widely used in cryptographic constructions such as **digital signatures, hash functions, and commitment schemes**.

#### **Problem Statement**:
<p align="center">
  <img src="./images/SIS.png" alt="image not in the same folder">
</p>

#### **Hardness**
- Finding a short solution requires solving a lattice shortest vector problem (SVP), which is NP-hard.
- If the entries of A are randomly chosen, then no efficient classical or quantum algorithm can solve SIS in general.
- The hardness of SIS is used to prove the security of various cryptographic schemes

#### **USES**
1. **Lattice-Based Digital Signatures:**
   
   - BLISS (Bimodal Lattice Signature Scheme)
   - Dilithium (NIST finalist for post-quantum cryptography)
  
2. **Hash Functions and Collision Resistance:**
   
   SIS provides a provably collision-resistant hash function:

   $H(x) = A.x (mod q)$

   If SIS is hard, finding two different inputs $x_1,x_2$ such that $H(x_1)=H(x_2)$ is infeasible.
3. **Zero-Knowledge Proofs & Commitments**

   SIS enables secure commitment schemes (e.g., Pedersen-like commitments in lattice settings).


In [13]:
def generate_sis_instance(n, m, q):
    # A * x ≡ 0 (mod q)
    A = random_matrix(ZZ, m, n, x=-q//2, y=q//2)
    return A % q

n, m, q = 5, 8, 97
A = generate_sis_instance(n, m, q) # random matrix of dimension (n,m) with each value (mod q)

In [ ]:
def sis_with_lll(A, q):
    # using LLL lattice reduction
    m, n = A.nrows(), A.ncols()
    
    B = A.augment(q * identity_matrix(m))
    # [ A | q*I ] <- augmented lattice basis
    
    B_red = B.LLL()

    # Extract a short solution (if it exists)
    for row in B_red.rows():
        if row[:n] != 0 and max(abs(e) for e in row[:n]) < q//2:  # Ensure small x
            return row[:n]

    print("no solution found.")
    return None

reduced_basis= sis_with_lll(A, q)

print("Matrix A:\n", A)
print("Short Integer Solution x:\n", reduced_basis)

# 3. Learning with Errors (LWE)

Learning with Errors (LWE) is a problem in lattice-based cryptography that forms the basis for many cryptographic protocols. It involves solving linear equations that are perturbed by small random errors.

#### Encryption
1. Choose a secret vector $\mathbf{s}$ from a uniform distribution.
2. Generate a random matrix $\mathbf{A} $ and a small error vector $\mathbf{e} $.
3. Compute the ciphertext as $\mathbf{c} = \mathbf{A} \mathbf{s} + \mathbf{e} $.

#### Decryption
1. Given the ciphertext $\mathbf{c} $ and the matrix $\mathbf{A} $, recover the secret vector $\mathbf{s} $ by solving the perturbed linear equations.
2. Use lattice reduction techniques to approximate the solution and remove the error vector $\mathbf{e} $.

<p align="center">
  <img src="./images/LWE.jpeg" alt="image not in the same folder">
</p>

In [ ]:
# To be run on Online sage cell server

from sage.all import *

# dimension
n = 64
# plaintext modulus
p = 257
# ciphertext modulus
q = 0x10001
# bound for error term
error_bound = int(floor((q/p)/2))
# message scaling factor
delta = int(round(q/p))

print("n = ", n)
print("p = ", p)
print("q = ", q)

m = 127


V = VectorSpace(GF(q), n)
S = V.random_element()

# Encryption
A = V.random_element()
error = randint(-error_bound, error_bound)
b = A * S + m * delta + error

print("A = ", A)
print("b = ", b)

# Decryption
x = int((b - A*S)%q)
m = round(x/delta)
print("m = ", m)


# 4. Knapsack Problem and Cryptanalysis

The Knapsack Problem is a well-known problem in combinatorial optimization. In cryptography, it has been used to create public-key cryptosystems. However, many knapsack-based cryptosystems have been broken using cryptanalysis techniques.

#### Knapsack Problem
Given a set of weights and a target sum, determine if there is a subset of weights that add up to the target sum.

#### Cryptanalysis
Cryptanalysis of knapsack-based cryptosystems often involves transforming the problem into a different domain where it becomes easier to solve. For example, lattice-based techniques can be used to break certain knapsack cryptosystems.

<p align="center">
  <img src="./images/knapsack.png" alt="image not in the same folder">
</p>

### Merkle–Hellman Knapsack Cryptosystem

The Merkle–Hellman knapsack cryptosystem is an early public-key cryptosystem that is based on the subset sum problem, a special case of the knapsack problem. It was invented by Ralph Merkle and Martin Hellman in 1978.

<p align="center">
  <img src="./images/Merkle-Hellman-knapsack-cryptosystem.png" alt="image not in the same folder">
</p>

#### Key Generation
1. **Superincreasing Sequence**: Generate a superincreasing sequence $ \mathbf{w} = (w_1, w_2, \ldots, w_n) $ where each element is greater than the sum of all previous elements.
2. **Modulus and Multiplier**: Choose a modulus $ M $ greater than the sum of all elements in $ \mathbf{w} $ and a multiplier $ k $ that is coprime with $M $.
3. **Public Key**: Compute the public key $ \mathbf{a} = (a_1, a_2, \ldots, a_n) $ where $a_i = (k \cdot w_i) \mod M $

**Encryption**: Given a public key $ \mathbf{a} = (a_1, a_2, \ldots, a_n) $ and a binary message $ \mathbf{m} = (m_1, m_2, \ldots, m_n) $, the ciphertext $ c $ is computed as:
   $
   c = \sum_{i=1}^{n} m_i a_i
   $

**Decryption**: Given the ciphertext $ c $, the private key (superincreasing knapsack) $ \mathbf{w} = (w_1, w_2, \ldots, w_n) $, the modulus $ M $, and the multiplier $ k $, the decryption process involves:
   - Computing the modular inverse of $ k $ modulo $ M $, denoted as $ k^{-1} $.
   - Calculating $ c' = (c \cdot k^{-1}) \mod M $.
   - Solving the subset sum problem using the superincreasing knapsack $ \mathbf{w} $ to recover the message $ \mathbf{m} $

#### Security and Cryptanalysis
The security of the Merkle–Hellman knapsack cryptosystem relies on the difficulty of solving the subset sum problem. However, it was broken by Adi Shamir in 1982 using lattice reduction techniques, which transformed the problem into a different domain where it became easier to solve. This cryptanalysis demonstrated that the Merkle–Hellman knapsack cryptosystem is not secure against certain types of attacks.


In [ ]:
# To be run on Online sage cell server

import numpy as np

# Superincreasing knapsack
def generate_superincreasing_knapsack(n):
    knapsack = [np.random.randint(1, 10)]
    for _ in range(1, n):
        knapsack.append(sum(knapsack) + np.random.randint(1, 10))
    return knapsack

# Modular knapsack
def generate_modular_knapsack(superincreasing_knapsack, modulus, multiplier):
    return [(multiplier * w) % modulus for w in superincreasing_knapsack]

# Encryption
def knapsack_encrypt(public_key, message):
    return sum(m * pk for m, pk in zip(message, public_key))

# Example usage
n = 8  # Length of the knapsack
superincreasing_knapsack = generate_superincreasing_knapsack(n)
modulus = sum(superincreasing_knapsack) + np.random.randint(1, 10)
multiplier = np.random.randint(2, modulus)
public_key = generate_modular_knapsack(superincreasing_knapsack, modulus, multiplier)

print("Public Key: ", public_key)

message = [1, 0, 1, 1, 0, 0, 1, 0]  # Example binary message
message_int = sum(m * 2**i for i, m in enumerate(message))
ciphertext = knapsack_encrypt(public_key, message)

print(f"Original message: {message_int}")
print(f"Ciphertext: {ciphertext}")

n = len(public_key)
N = ceil(sqrt(n) / 2)

b = []
for i in range(n):
    vec = [0 for _ in range(n + 1)]
    vec[i] = 1
    vec[-1] = N * public_key[i]
    b.append(vec)

b.append([1 / 2 for _ in range(n)] + [N * ciphertext])

BB = matrix(QQ, b)
l_sol = BB.LLL()

for e in l_sol:
    if e[-1] == 0:
        msg = 0
        isValidMsg = True
        for i in range(len(e) - 1):
            ei = 1 - (e[i] + (1 / 2))
            if ei != 1 and ei != 0:
                isValidMsg = False
                break

            msg |= int(ei) << i

        if isValidMsg:
            print('Original message decrypted:', msg)
            break